# Zillow ZPrediction Challenge

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

PATH='data/zillow/'

In [3]:
from IPython.display import HTML

In [16]:
train = pd.read_csv(f'{PATH}train_2016_v2.csv', low_memory=False)
properties = pd.read_csv(f'{PATH}properties_2016.csv', low_memory=False)

In [7]:
train.head()

,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01
2,12098116,-0.0040,2016-01-01
3,12643413,0.0218,2016-01-02
4,14432541,-0.0050,2016-01-02


In [5]:
properties.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


In [6]:
display(DataFrameSummary(properties).summary())

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
count,2.98522e+06,811519,6061,1628,2.97376e+06,2.97377e+06,12629,1.93849e+06,2.85630e+06,17096,...,682069,NaN,2.93024e+06,2.94267e+06,2.97378e+06,2.91748e+06,2.95397e+06,NaN,56464,2.91009e+06
mean,1.33259e+07,1.93117,7.20261,646.883,2.20914,3.08895,3.72595,5.78479,2.29926,66,...,1.40146,NaN,170884,420479,2015,252478,5377.61,NaN,13.8924,6.04843e+13
std,7.90997e+06,3.14859,2.43629,538.793,1.07775,1.27586,0.5017,1.80535,1.00074,0,...,0.539076,NaN,402068,726347,0.0368316,445013,9183.11,NaN,2.58101,3.24903e+11
min,1.07117e+07,1,2,20,0,0,1,1,1,66,...,1,NaN,1,1,2000,1,1.34,NaN,0,-1
25%,1.16437e+07,1,7,272,2,2,3,4,2,66,...,1,NaN,74800,179675,2015,74836,2461.07,NaN,14,6.0374e+13
50%,1.25451e+07,1,7,534,2,3,4,7,2,66,...,1,NaN,122590,306086,2015,167042,3991.78,NaN,14,6.03757e+13
75%,1.40971e+07,1,7,847.25,3,4,4,7,3,66,...,2,NaN,196889,488000,2015,306918,6201.01,NaN,15,6.05904e+13
max,1.69602e+08,13,27,8516,20,20,5,12,20,66,...,41,NaN,2.51486e+08,2.82786e+08,2016,9.02462e+07,3.45886e+06,NaN,99,4.8303e+14
counts,2985217,811519,6061,1628,2973755,2973767,12629,1938488,2856305,17096,...,682069,5163,2930235,2942667,2973778,2917484,2953967,56462,56464,2910091
uniques,2985217,7,8,751,37,21,5,12,34,1,...,12,1,426150,638920,14,531408,1045757,1,32,96771


In [12]:
sample_submission = pd.read_csv(f'{PATH}sample_submission.csv', low_memory=False)

In [9]:
sample_submission.head()

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0,0,0,0,0,0
1,10759547,0,0,0,0,0,0
2,10843547,0,0,0,0,0,0
3,10859147,0,0,0,0,0,0
4,10879947,0,0,0,0,0,0


In [18]:
??ColumnarModelData.from_data_frame